In [1]:
# Установка необходимых пакетов
!pip install pandas numpy nltk pymorphy2 scikit-learn keras imbalanced-learn joblib matplotlib seaborn pymorphy2-dicts-ru
!pip install tensorflow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=94785d27d41828a2d5d95c11645923ffd83664477bf1625b7b16f0afc159d990
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
import os
import zipfile
from joblib import load
from keras.models import load_model
import pymorphy2
from nltk.corpus import stopwords
import numpy as np
from scipy.sparse import hstack
import nltk
import re
zip_filename = 'model_weights (1).zip'


print(f"Загрузка моделей и объектов из архива {zip_filename}...")

# Проверка существования архива
if not os.path.exists(zip_filename):
    raise FileNotFoundError(f"Архив {zip_filename} не найден.")

# Распаковка архива
with zipfile.ZipFile(zip_filename, 'r') as zf:
    zf.extractall()
    print(f"Архив {zip_filename} распакован.")

Загрузка моделей и объектов из архива model_weights (1).zip...
Архив model_weights (1).zip распакован.


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# Загрузка модели и объектов
rf_model = load('random_forest_model.joblib')
print("Загружена модель Random Forest.")

vectorizer = load('tfidf_vectorizer.joblib')
print("Загружен TfidfVectorizer.")

scaler = load('standard_scaler.joblib')
print("Загружен StandardScaler.")

le = load('label_encoder.joblib')
print("Загружен LabelEncoder.")

print("Все модели и объекты загружены.")

# Инициализация инструментов для предобработки текста
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def tokenize_and_lemmatize(text, morph, stop_words):
    # Удаление чисел и специальных символов
    text = re.sub(r'[^а-яА-Я\s]', '', str(text))
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(tokens)

# Определение функции для классификации новых контрактов
def classify_contract(text, contract_value=0, contract_duration=0):
    # Предобработка входного текста
    processed_text = tokenize_and_lemmatize(text, morph, stop_words)
    text_vector = vectorizer.transform([processed_text])

    # Масштабирование числовых признаков
    contract_value_scaled = scaler.transform([[contract_value]])
    contract_duration_scaled = scaler.transform([[contract_duration]])
    numerical_features = np.array([[contract_value_scaled[0][0], contract_duration_scaled[0][0]]])

    X_input = hstack([text_vector, numerical_features])

    # Предсказание с помощью Случайного Леса
    pred_rf = rf_model.predict(X_input)
    pred_rf_label = le.inverse_transform(pred_rf)[0]

    # Вывод результата
    print(f"\nВходной текст: {text}\n")
    print(f"Предсказание Случайного Леса: {pred_rf_label}")


# Пример классификации
input_text = input("Введите описание контракта для классификации: ")
try:
    contract_value_input = input("Введите сумму контракта (или нажмите Enter для установки в 0): ")
    contract_value = float(contract_value_input) if contract_value_input.strip() != "" else 0.0
except ValueError:
    print("Некорректное значение. Установка в 0.")
    contract_value = 0.0

try:
    contract_duration_input = input("Введите продолжительность контракта в днях (или нажмите Enter для установки в 0): ")
    contract_duration = float(contract_duration_input) if contract_duration_input.strip() != "" else 0.0
except ValueError:
    print("Некорректное значение. Установка в 0.")
    contract_duration = 0.0

classify_contract(input_text, contract_value, contract_duration)


Загружена модель Random Forest.
Загружен TfidfVectorizer.
Загружен StandardScaler.
Загружен LabelEncoder.
Все модели и объекты загружены.
Введите описание контракта для классификации: Земляные работы. Как правило, это работы, которые напрямую связаны с необходимостью подготовки строительной площадки. К примеру, рытье траншей и котлованов для обустройства фундаментов или монтажа подземных коммуникаций. 
Введите сумму контракта (или нажмите Enter для установки в 0): 1680000
Введите продолжительность контракта в днях (или нажмите Enter для установки в 0): 20

Входной текст: Земляные работы. Как правило, это работы, которые напрямую связаны с необходимостью подготовки строительной площадки. К примеру, рытье траншей и котлованов для обустройства фундаментов или монтажа подземных коммуникаций. 

Предсказание Случайного Леса: СМР


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


-